In [1]:
import os
from pathlib import Path
os.chdir(Path("D:/Sagar_office/repo/chicken-disease-classification-Project"))

In [2]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
# cofiguration manager
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

# config manager
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir), 
            Path(config.tensorboard_root_log_dir)
            ])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
           root_dir = Path(config.root_dir),
           tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
           checkpoint_model_filepath = Path(config.checkpoint_model_filepath)     
        )

        return prepare_callbacks_config

In [4]:
# create components
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_runninng_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_logs_at_{timestamp}"
            )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_runninng_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        print(self.config.checkpoint_model_filepath)
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [ ]:
# prepare pipeline
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    # cllback_list will give error if not worked with model training
except Exception as e:
   raise e

[2024-10-31 07:37:15,236: INFO: yaml file: config\config.yaml loaded successfully]
[2024-10-31 07:37:15,242: INFO: yaml file: params.yaml loaded successfully]
[2024-10-31 07:37:15,244: INFO: create directory at: artifacts]
[2024-10-31 07:37:15,248: INFO: create directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-10-31 07:37:15,251: INFO: create directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
artifacts\prepare_callbacks\checkpoint_dir\model.keras


AttributeError: 'WindowsPath' object has no attribute 'endswith'